In [1]:
import tkinter as tk
from tkinter import ttk
from pymongo import MongoClient
from joblib import load
import pandas as pd
import numpy as np
import webbrowser
from sklearn.metrics import pairwise_distances


In [2]:
client = MongoClient('mongodb://localhost:27017/')
db = client['GameDB']
collection = db['Games_final']
df = pd.DataFrame(list(collection.find()))
collection_lft = db['Games']
df_additional_tags = pd.DataFrame(list(collection_lft.find()))
collection_info = db['GameInfo']
df_info = pd.DataFrame(list(collection_info.find()))
model = load('Models/model_kmeans_0.5540.joblib')
df_merged = pd.merge(df, df_info[['AppID', 'Positive',"Negative"]], on='AppID', how='left')
df["NetVotes"] = df_merged["Positive"]-df_merged["Negative"]
df["NetVotes"] = (df["NetVotes"]- df["NetVotes"].min()) / (df["NetVotes"].max() - df["NetVotes"].min())
df_additional_tags = df_additional_tags.drop(['action', 'adventure', 'role-playing games', 'simulation',
       'strategy'],axis=1)
df_additional_tags = df_additional_tags.drop(['_id', 'Net Votes', 'Peak CCU'],axis=1)

#### Safe to use ( no NSFW results ) appids :
Hogwards Legacy 990080 ,Trove 304050, FIFA24 2195250,

In [5]:
def get_game_details(appid):
    game = db.GameInfo.find_one({"AppID": appid})
    if game == None:
        return 0
    return game 

def get_recommendations(appid):
    #Check if game is in the dataset
    if appid in df['AppID'].values:
        input_game = df[df['AppID'] == appid]
        input_game_full_tags =  pd.merge(input_game, df_additional_tags, on='AppID').drop(["cluster","AppID","_id","NetVotes"],axis=1)
        input_game_info = df_info[df_info['AppID']==appid]
        input_game_cluster = input_game['cluster'].iloc[0]
        input_game_votes = input_game['NetVotes'].iloc[0]
        #Check if a game was inserted with later scraping, if it has no cluster (Nan), predict it
        if input_game_cluster == None:
            print("Game not clustered, assigning cluster")
            input_game_cluster = model.predict(input_game)[0]
        input_game = input_game.drop(['cluster','AppID','NetVotes'],axis=1)
        #Get games from the same cluster
        recommended_df  = df[df['cluster'] == input_game_cluster].copy()
        #Get additional low frequency tags
        recommended_df = pd.merge(recommended_df, df_additional_tags, on='AppID')

        #Remove the input game from the recommendations
        recommended_df = recommended_df[recommended_df['AppID'] != appid]
        #Remove non-binary attributes
        appids = recommended_df['AppID']
        netvotes = recommended_df['NetVotes']
        recommended_df=recommended_df.drop(['_id','cluster','AppID','NetVotes'],axis = 1)
        #Compute jaccard distance matrix
        print(input_game_full_tags.columns)
        print(recommended_df.columns)
        distances = pairwise_distances(recommended_df.to_numpy(), input_game_full_tags.to_numpy(), metric='jaccard').flatten()
        #Compute similarity matrix
        similarities = 1 - distances
        recommended_df['Similarity'] = similarities
        recommended_df['AppID'] = appids
        w1, w2 = 1, 0.073
        recommended_df['Weighted Similarity'] = (w1 * recommended_df['Similarity'] + 
                                                    w2 * netvotes )
        #Get the most similar games from the same cluster
        closest_games = recommended_df.sort_values(by='Weighted Similarity', ascending=False)
        closest_games = closest_games.head(100)
        #Get the results informations and return them
        results = pd.DataFrame()

        for g in closest_games['AppID']:
            # Filter df_info for the current AppID
            tmp = df_info[df_info['AppID'] == g]
            # Concatenate tmp with the accumulating results DataFrame
            results = pd.concat([results, tmp], ignore_index=True)

        return results
    else:
        #The game is not in the database, if the game does exist, launch the scraping process offline
        return 0

def on_recommend():
    appid = int(app_id_entry.get())
    input_game = get_game_details(appid)
    if input_game == 0:
        popup_message("Game not in DB","The game is not in the DB, if the AppID exists, launch the scraping code offline to update the system")
        return None
    input_game_info.config(text=f"Name: {input_game['Name']}\nGenres: {input_game['Genres']}")
    try:
        recommended_games = get_recommendations(appid)
    except Exception as e:
        print(f"Error during recommendation: {e}")
        
    for i in tree.get_children():
        tree.delete(i)

    for _, game in recommended_games.iterrows():
        genres = game['Genres']
        if not isinstance(genres, list):
            genres = genres.split(',') 
        tree.insert('', 'end', values=(game['AppID'], game['Name'], ', '.join(genres),game['Positive'],game['Negative'],game['Peak CCU'] ))


def on_double_click(event):
    item_id = tree.selection()[0]
    item = tree.item(item_id)
    appid = item['values'][0]
    url = f"https://store.steampowered.com/app/{appid}"
    webbrowser.open(url)


def popup_message(title,message):
    popup = tk.Toplevel()
    popup.title(title)
    message_label = tk.Label(popup, text=message, font=('Calibri', 16))
    message_label.pack(pady=(10, 0), padx=10)
    okay_button = ttk.Button(popup, text="Ok", command=popup.destroy)
    okay_button.pack(pady=(5, 10), padx=10)
    popup.transient(root)  
    popup.grab_set()  
    root.wait_window(popup) 


In [ ]:
from ctypes import windll
windll.shcore.SetProcessDpiAwareness(1)

root = tk.Tk()
root.title("Game Recommendation System")
root.state('zoomed')


style = ttk.Style()
style.theme_use("clam")  # This is a more modern theme than the default

# Customize the Treeview colors
style.configure("Treeview",
                background="#E8E8E8",
                foreground="black",
                rowheight=25,
                fieldbackground="#E8E8E8")
style.map('Treeview', background=[('selected', '#3498db')])

# Customize the Treeview Header
style.configure("Treeview.Heading", font=('Calibri', 16, 'bold'))



#Input textbopx
input_frame = tk.Frame(root, bg='#2c3e50', pady=5)
input_frame.pack(fill='x')

#Input Game Info
input_game_info = tk.Label(root, text="Input Game", bg='#2c3e50', fg='white', font=('Calibri', 16))
input_game_info.pack(fill='x')
# AppID Entry
app_id_label = tk.Label(input_frame, text="Enter Game AppID:", bg='#2c3e50', fg='white', font=('Calibri', 16))
app_id_label.pack(side='left', padx=10)
app_id_entry = tk.Entry(input_frame, font=('Calibri', 14))  
app_id_entry.pack(side='left', fill='x', expand=True, padx=10)


# Recommend Button
recommend_button = tk.Button(input_frame, text="Recommend", font=('Calibri', 16, 'bold'), command=on_recommend)
recommend_button.pack(side='left', padx=10)


# Treeview Frame
tree_frame = tk.Frame(root)
tree_frame.pack(fill='both', expand=True)

# Add a scrollbar to the Treeview
tree_scroll = tk.Scrollbar(tree_frame)
tree_scroll.pack(side='right', fill='y')


columns = ('AppID', 'Name', 'Genres', 'Positive', 'Negative', 'Peak CCU')
tree = ttk.Treeview(tree_frame, yscrollcommand=tree_scroll.set, columns=columns, show='headings', selectmode='browse')

# Configure the scrollbar
tree_scroll.config(command=tree.yview)
tree.heading('AppID', text='AppID')
tree.heading('Name', text='Name')
tree.heading('Genres', text='Genres')
tree.heading('Positive', text='Positive Reviews')
tree.heading('Negative', text='Negative Reviews')
tree.heading('Peak CCU', text='Peak CCU')

tree.column('AppID', width=100, anchor='center')
tree.column('Name', width=200, anchor='center')
tree.column('Genres', width=300, anchor='center')
tree.column('Positive', width=100, anchor='center')
tree.column('Negative', width=100, anchor='center')
tree.column('Peak CCU', width=100, anchor='center')
tree.bind("<Double-1>", on_double_click)

# Pack the Treeview into the GUI
tree.pack(side='left', fill='both', expand=True)
root.mainloop()


Index(['action', 'adventure', 'role-playing games', 'simulation', 'strategy',
       'animation & modeling', 'audio production', 'casual',
       'design & illustration', 'early access', 'education', 'free to play',
       'game development', 'gore', 'indie', 'mature content', 'mmo', 'nudity',
       'photo editing', 'racing', 'software training', 'sports', 'utilities',
       'violent', 'web publishing'],
      dtype='object')
Index(['action', 'adventure', 'role-playing games', 'simulation', 'strategy',
       'animation & modeling', 'audio production', 'casual',
       'design & illustration', 'early access', 'education', 'free to play',
       'game development', 'gore', 'indie', 'mature content', 'mmo', 'nudity',
       'photo editing', 'racing', 'software training', 'sports', 'utilities',
       'violent', 'web publishing'],
      dtype='object')


C:\Users\senti\anaconda3\Lib\site-packages\sklearn\metrics\pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


Index(['action', 'adventure', 'role-playing games', 'simulation', 'strategy',
       'animation & modeling', 'audio production', 'casual',
       'design & illustration', 'early access', 'education', 'free to play',
       'game development', 'gore', 'indie', 'mature content', 'mmo', 'nudity',
       'photo editing', 'racing', 'software training', 'sports', 'utilities',
       'violent', 'web publishing'],
      dtype='object')
Index(['action', 'adventure', 'role-playing games', 'simulation', 'strategy',
       'animation & modeling', 'audio production', 'casual',
       'design & illustration', 'early access', 'education', 'free to play',
       'game development', 'gore', 'indie', 'mature content', 'mmo', 'nudity',
       'photo editing', 'racing', 'software training', 'sports', 'utilities',
       'violent', 'web publishing'],
      dtype='object')


C:\Users\senti\anaconda3\Lib\site-packages\sklearn\metrics\pairwise.py:2025: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)
